In [1]:
import numpy as np
import math

class Vector(object):
    
    CANNOT_NORMALISE_ZERO_VECTOR_MSG = 'Cannot normalise the zero vector'
    
    def __init__(self, coordinates):
        try:
            if not coordinates:
                raise ValueError
            self.coordinates = tuple(coordinates)
            self.dimension = len(coordinates)

        except ValueError:
            raise ValueError('The coordinates must be nonempty')

        except TypeError:
            raise TypeError('The coordinates must be an iterable')


    def __str__(self):
        return 'Vector: {}'.format(self.coordinates)


    def __eq__(self, v):
        return self.coordinates == v.coordinates
    
    def __add__(self, v):
        if self.dimension == v.dimension :
            return [x + y for x, y in zip(self.coordinates, v.coordinates)]
        else :
            return
        
    def __sub__(self, v):
        if self.dimension == v.dimension :
            return [x - y for x, y in zip(self.coordinates, v.coordinates)]
        else :
            return
    
    def scalar_mul(self, s):
        return [s*x for x in self.coordinates]
    
    def magnitude(self):
        return sum([x**2 for x in self.coordinates])**(.5)
    
    def normalise(self):
        try :
            return [(1./self.magnitude())*x for x in self.coordinates]
        except ZeroDivisionError:
            raise Exception('Cannot normalise zero vector')
        
    def dot(self, v):
        return sum([x*y for x, y in zip(self.coordinates, v.coordinates)])
    
    def angle(self, v, in_degrees=False):
        if in_degrees:
            return math.degrees(math.acos(self.dot(v)/(self.magnitude() * v.magnitude())))
        else:    
            #angle = round(self.dot(v)/(self.magnitude() * v.magnitude()),15)
            u1 = Vector(self.normalise())
            u2 = Vector(v.normalise())
            angle_in_radians = math.acos(u1.dot(u2))
            return angle_in_radians
        #         try :
        #             u1 = self.normalise()
        #             u2 = v.normalise()
        #             angle_in_radians = math.acos(u1.dot(u2))

        #             if in_degrees:
        #                 degrees_per_radian = 180. / math.pi
        #                 return angle_in_radians * degrees_per_radian
        #             else:
        #                 return angle_in_radians

        #         except Exception as e:
        #             if str(e) == self.CANNOT_NORMALISE_ZERO_VECTOR_MSG:
        #                 raise Exception('Cannot computer an angle with the zero vector')
        #             else:
        #                 raise e

    def orthogonal(self, v, tolerance=1e-10):
        return abs(self.dot(v)) < tolerance

    def is_zero(self, tolerance=1e-10):
        return self.magnitude() < tolerance

    def parallel(self, v):
        return (self.is_zero() or v.is_zero() or self.angle(v)==0 or self.angle(v)==math.pi)
    
    def projection(self, b):
        return Vector(b.normalise()).scalar_mul(self.dot(Vector(b.normalise())))
    
    def component_orthogonal_to(self, b):
        projection = Vector(self.projection(b))
        return self - projection
    
    def cross_product(self, v):
        a = self.coordinates
        b = v.coordinates    
        if  self.dimension == 2 and v.dimension == 2:
            # add 0 for the z dimension
            a = a + (0,)
            b = b + (0,)
        
        return [ a[1]*b[2] - b[1]*a[2], -(a[0]*b[2] - b[0]*a[2]), a[0]*b[1] - b[0]*a[1] ]
    
    def parallelogram_area(self, v):
        return sum([ x**2 for x in self.cross_product(v)])**(1./2)
    
    def triangle_area(self, v):
        return self.parallelogram_area(v)*(1./2)

    
my_vector = Vector([1,2])
print my_vector

Vector: (1, 2)


In [3]:
from decimal import Decimal, getcontext


getcontext().prec = 30


class Line(object):

    NO_NONZERO_ELTS_FOUND_MSG = 'No nonzero elements found'

    def __init__(self, normal_vector=None, constant_term=None):
        self.dimension = 2

        if not normal_vector:
            all_zeros = ['0']*self.dimension
            normal_vector = Vector(all_zeros)
        self.normal_vector = normal_vector

        if not constant_term:
            constant_term = Decimal('0')
        self.constant_term = Decimal(constant_term)

        self.set_basepoint()


    def set_basepoint(self):
        try:
            n = self.normal_vector
            c = self.constant_term
            basepoint_coords = ['0']*self.dimension
            print basepoint_coords

            initial_index = Line.first_nonzero_index(n)
            print initial_index
            initial_coefficient = n[initial_index]
            print initial_coefficient

            basepoint_coords[initial_index] = c/initial_coefficient
            print basepoint_coords
            self.basepoint = Vector(basepoint_coords)
            print self

        except Exception as e:
            if str(e) == Line.NO_NONZERO_ELTS_FOUND_MSG:
                self.basepoint = None
            else:
                raise e


    def __str__(self):

        num_decimal_places = 3

        def write_coefficient(coefficient, is_initial_term=False):
            coefficient = round(coefficient, num_decimal_places)
            if coefficient % 1 == 0:
                coefficient = int(coefficient)

            output = ''

            if coefficient < 0:
                output += '-'
            if coefficient > 0 and not is_initial_term:
                output += '+'

            if not is_initial_term:
                output += ' '

            if abs(coefficient) != 1:
                output += '{}'.format(abs(coefficient))

            return output

        n = self.normal_vector

        try:
            initial_index = Line.first_nonzero_index(n)
            terms = [write_coefficient(n[i], is_initial_term=(i==initial_index)) + 'x_{}'.format(i+1)
                     for i in range(self.dimension) if round(n[i], num_decimal_places) != 0]
            output = ' '.join(terms)

        except Exception as e:
            if str(e) == self.NO_NONZERO_ELTS_FOUND_MSG:
                output = '0'
            else:
                raise e

        constant = round(self.constant_term, num_decimal_places)
        if constant % 1 == 0:
            constant = int(constant)
        output += ' = {}'.format(constant)

        return output


    @staticmethod
    def first_nonzero_index(iterable):
        for k, item in enumerate(iterable):
            if not MyDecimal(item).is_near_zero():
                return k
        raise Exception(Line.NO_NONZERO_ELTS_FOUND_MSG)


class MyDecimal(Decimal):
    def is_near_zero(self, eps=1e-10):
        return abs(self) < eps


l1 = Line(normal_vector=Vector(['4.046','2.836']), constant_term='1.21')
l2 = Line(normal_vector=Vector(['10.115','7.09']), constant_term='3.025')

print l1
print l2

['0', '0']


TypeError: 'Vector' object is not iterable

In [175]:
l1 = Line(normal_vector=Vector(['4.046','2.836']), constant_term='1.21')
l2 = Line(normal_vector=Vector(['10.115','7.09']), constant_term='3.025')
# print l1
# print l2
# print l1.basepoint
# print l2.basepoint
# print l1.constant_term
# print l1.dimension
# print l1.normal_vector
# print l2.normal_vector

TypeError: 'Vector' object is not iterable